In [75]:
%run '/home/lena/Desktop/Application Project/src/ibm1.py'
%run '/home/lena/Desktop/Application Project/src/ibm2.py'
%run '/home/lena/Desktop/Application Project/src/pbmt.py'
%run '/home/lena/Desktop/Application Project/src/utils.py'
%run '/home/lena/Desktop/Application Project/src/test.py'
import numpy as np

In [90]:
TEST = True 
nr_used_sentences = 100 #1800 for 1model,25steps  # of 688670 lines
path = "./src/Paralel Corpus/"
e, f = read_in_corpus(nr_used_sentences, path)

training_split = 0.9
e_train, e_test = split_dataset(e, training_split)
f_train, f_test = split_dataset(f, training_split)
if TEST:
    e_train, f_train, e_test, f_test, all_alignments_test = initialize_test_sets()
#TODO or save alignments as { 0: {0:[0]}, 1: {0:[0,1], 1:[3], 2:[1], 3:[2]} } ?

max_steps = 25

t_e2f = EM_IBM_Model_1(e_train, f_train, max_steps)
#TODO save t, a in files & load them?

compare_ibm_1(t_e2f, max_steps, e_train, f_train)  # to test

# TODO my a is 1 when it should be low :(
t, a = EM_IBM_Model_2(e_train, f_train, t_e2f, max_steps)
#compare_ibm_2(t, max_steps, a, e_train, f_train)  # to test
print('------')
t_f2e = EM_IBM_Model_1(f_train, e_train, max_steps)
#t, a = EM_IBM_Model_2(f_train, e_train, t_f2e, max_steps)
#compare_ibm_2(t, max_steps, a, f_train, e_train)

sum = 0
for i in range(len(f_test)):
    if (i,4,2,2) in a:
        sum += a[(i,4,2,2)]
   # else:
    #    print('tüdelü')
#print(sum,f_test[i],e_test[4],' TODO sum is supposed to always be 1')

# Testing
#TODO p(e|f) calc is wrong!!!!
wrong1 = 0
wrong2 = 0
for i in range(0, len(f_test)):
    f = f_test[i]
    e = e_test[i]
    p1 = prob_e_given_f_1(e, f, 1, t_e2f)
    if (p1 < 0.01):
        wrong1 += 1
    print('p(', e, '|', f, ') =', p1)
    p2 = 0#prob_e_given_f_2(e, f, 1, t, a)
    if (p2 < 0.5):
        wrong2 += 1
    #print('a: p(', e, '|', f, ') =', p2)
    
print('IBM Model 1 :', int(100*(1-wrong1/len(f_test))),'% correct')
print('IBM Model 2 :', int(100*(1-wrong2/len(f_test))),'% correct')
        
# arg max_e p(e)p(f|e) decoding manually #TODO how???
# calculate p(e|f)

wrong t house Haus 0.666666296495824 0.9999991271302255
------
p( ['the'] | ['das'] ) = 1.0
p( ['the', 'the', 'house', 'a', 'book'] | ['das', 'ein', 'Buch', 'Haus'] ) = 0.0011531999841871323
p( ['book', 'a'] | ['Buch', 'ein'] ) = 0.24999999997398636
p( ['book', 'the'] | ['Buch', 'ein'] ) = 2.601366715804536e-11
p( ['kgefw', 'fek'] | ['Buch', 'ein'] ) = 0.0
IBM Model 1 : 40 % correct
IBM Model 2 : 0 % correct


In [127]:
# Phrase - based MT
def viterbi_alignment(src, tar, t):#, a):
    #TODO what if no alignment or (src_j, tar_i) not in t?
    #return dictionary viterbi_alignment src -> tar
    viterbi_alignment = {}
    l_e = len(src)
    l_f = len(tar)
    for (j, src_j) in enumerate(src):
        max_value = -1
        max_index = None
        for (i, tar_i) in enumerate(tar):
            if (src_j, tar_i) in t:
                value = t[(src_j, tar_i)] #* a[(i, j, l_e, l_f)]
                # select the first one among the maximum candidates
                if max_value < value:
                    max_value = value
                    max_index = i
        viterbi_alignment[j] = max_index
    return viterbi_alignment

e_test = [['the'],['the','the','house','a','book'],['book','a'],['book','the'],['kgefw', 'fek']]
f_test = [['das'],['das','ein','Buch','Haus'],['Buch','ein'],['Buch','ein'],['Buch','ein']]
all_alignments_test = {0:[[1]], 1:[[1,0,0,0],[1,0,0,0],[0,0,0,1],[0,1,0,0],[0,0,1,0]], 2:[[1,0],[0,1]], 3:[[1,0],[0,1]], 4:[[1,0],[0,1]]}
    
def inter(e2f, f2e):
    intersection = {}
    for j in range(len(e2f)):
        i = e2f[j]
        if i != None and j == f2e[i]:
            intersection[j] = i
    return intersection

def unio(e2f, f2e):
    union = {}
    for j in range(len(e2f)):
        i = e2f[j]
        if i != None:
            union[j] = union.get(j,[])
            union[j].append(i)
    for i in range(len(f2e)):
        j = f2e[i]
        if j != None:
            union[j] = union.get(j,[])
            if i not in union[j]: union[j].append(i)
    return union

def word_alignment(all_e2f, allf2e):
    all_e2f ={}
    all_f2e ={}
    for si in range(0,len(f_test)):
        e2f = viterbi_alignment(e_test[si], f_test[si], t_e2f)
        f2e = viterbi_alignment(f_test[si], e_test[si], t_f2e)
        print(e_test[si],f_test[si])
        print('e2f',e2f)
        print('f2e',f2e)
        print('inter',inter(f2e, e2f)) #(e2f, f2e)
        print('unio', unio(f2e, e2f))
        print('-----')
        all_e2f[si] = e2f
        all_f2e[si] = f2e
word_alignment(all_e2f, all_f2e)
#In my first approach I had implemented the word alignment using the alignment matrices as in the lecture. That turned out as a reason for unnecessary big memory usage and so I decided instead to use a dict as a datastructure with a mapping from index to list of indices.

['the'] ['das']
e2f {0: 0}
f2e {0: 0}
inter {0: 0}
unio {0: [0]}
-----
['the', 'the', 'house', 'a', 'book'] ['das', 'ein', 'Buch', 'Haus']
e2f {0: 0, 1: 0, 2: 3, 3: 1, 4: 2}
f2e {0: 0, 1: 3, 2: 4, 3: 2}
inter {0: 0, 1: 3, 2: 4, 3: 2}
unio {0: [0, 1], 1: [3], 2: [4], 3: [2]}
-----
['book', 'a'] ['Buch', 'ein']
e2f {0: 0, 1: 1}
f2e {0: 0, 1: 1}
inter {0: 0, 1: 1}
unio {0: [0], 1: [1]}
-----
['book', 'the'] ['Buch', 'ein']
e2f {0: 0, 1: 0}
f2e {0: 0, 1: 0}
inter {0: 0}
unio {0: [0, 1], 1: [0]}
-----
['kgefw', 'fek'] ['Buch', 'ein']
e2f {0: None, 1: None}
f2e {0: None, 1: None}
inter {}
unio {}
-----


In [92]:
# Phrase - based MT

#e2f = ibmmodel2.viterbi_alignment(fs, es, *e2f_train).items()
#TODO interesting here: f, e but e2f

import collections
# Viterbi alignment
# find the most probable alignment for a given translation pair
def viterbi_alignment(es, fs, t):#, a):
    #return dictionary e in es -> f in fs
    max_a = collections.defaultdict(float)
    l_e = len(es)
    l_f = len(fs)
    for (j, e) in enumerate(es, 1):
        current_max = (0, -1)
        for (i, f) in enumerate(fs, 1):
            if (e,f) in t:
                val = t[(e, f)] #* a[(i, j, l_e, l_f)]
                # select the first one among the maximum candidates
                if current_max[1] < val:
                    current_max = (i, val)
        max_a[j] = current_max[0]
    return max_a

# IBM Models for word alignment
# TODO I have to use ibm model 2!!!!
# TODO actually use e_train but here e_test only to test
#all_aligned_e2f = align(e_train, f_train, t_e2f)
#all_aligned_f2e = align(f_train, e_train, t_f2e)

all_aligned_e2f = align(e_test, f_test, t_e2f)
all_aligned_f2e = align(f_test, e_test, t_f2e)
for si in range(0,len(f_test)):
    new_all_aligned_e2f = viterbi_alignment(e_test[si], f_test[si], t_e2f)
    new_all_aligned_f2e = viterbi_alignment(f_test[si], e_test[si], t_f2e)
    print(e_test[si],f_test[si],'\nviterbi e2f:',new_all_aligned_e2f,'\nviterbi:',new_all_aligned_f2e)
    print('my f2e:', all_aligned_f2e[si],'\n my e2f:', all_aligned_e2f[si], '\nright:',all_alignments_test[si],'\n-----')
#align = alignment(es, fs, e2f, f2e)  # symmetrized alignment

# transpose so that alignments are same shape
all_aligned_f2e = [np.transpose(f2e) for f2e in list(all_aligned_f2e.values())]
#for sentence in range(0, len(e_train)):
	#print(e_train[sentence], f_train[sentence])
	#print(all_aligned_e2f[sentence])
	#print(all_aligned_f2e[sentence])
# x foreign, y English

# call for every sentence pair
all_alignments = {}
for index in range(0, len(e_test)):
	e2f = all_aligned_e2f[index]
	f2e = all_aligned_f2e[index]
	alignment = grow_diag_final(e2f, f2e)
	#print(alignment)
	all_alignments[index] = alignment
    
for index in range(len(all_alignments)):
    print(all_alignments[index].astype(int) == all_alignments_test[index])
    print('compare aligns',all_alignments[index], all_alignments_test[index])

######
"""
e_train = [['michael','assumes','that','he','will','stay','in','the','house']]
f_train = [['michael','geht','davon','aus',',','dass','er','im','haus','bleibt']]
all_alignments = {0:[[1,0,0,0,0,0,0,0,0,0],
                   [0,1,1,1,0,0,0,0,0,0],
                   [0,0,0,0,0,1,0,0,0,0],
                   [0,0,0,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,0,0,0,1],
                   [0,0,0,0,0,0,0,0,0,1],
                   [0,0,0,0,0,0,0,1,0,0],
                   [0,0,0,0,0,0,0,1,0,0],
                   [0,0,0,0,0,0,0,0,1,0]]}
# all_alignments[a][b] alignment for b word in e for sentence a
"""

#all_BP = phrase_extraction(e_train, f_train, all_alignments)
# for BP in all_BP:
#print(all_BP)

# probabilistic phrase translation table
# For each sentence pair, we extract a number of phrase pairs.
# Then, we count in how many sentence pairs a particular phrase pair is
# extracted and store this number in count(e,f).
all_BP_flat = [item for sublist in all_BP for item in sublist]
count = {}
for (a, b) in all_BP_flat:
	# key = (frozenset(a),frozenset(b))
	key = str((a, b))
	count[key] = count.get(key, 0) + 1
print(count)

#print(φ(['house'], ['Haus'], count))
#print(φ(['the','house'],['das','Haus'],count))

# TODO bigram language model
#smoothing in case of 0
#just table of p(a|b) for all word combinations? train on f_trai, e_train
# eg -0.2359 When will

# Sentence Alignments in both directions with Viterbi algorithm using IBM Model 2
# While it is true that during the EM training fractional counts are
# collected over a probability distribution of possible alignments, one
# word alignment sticks out: the most probable alignment, also called the
# Viterbi alignment
# > output the Viterbi alignment of the last iteration
# > run IBM model training in both directions

# TODO phrase-based statistical machine translation model
# • the phrase translation table φ(f |e);
# • the reordering model d;
# • the language model pLM(e).
# e_best = argmax_e prod_i φ(f|e) * d(start_i - end_i_min_1 - 1) * prod_i PLM(e_i|e_i_min_1)
for sentence in range(0, len(all_alignments_test)):
    f = f_test[sentence]
    e = e_test[sentence]
    alignment = all_alignments_test[sentence]
    for j in range(0, len(e)):
        e_j = e[j]
        start_j, end_j_min_1 = calc_vals_for_d(alignment, j, f)
        d(start_j - end_j_min_1 - 1)
        #to test reordering
        #print(j, start_j, end_j_min_1, start_j - end_j_min_1 - 1)


['the'] ['das'] 
viterbi e2f: defaultdict(<class 'float'>, {1: 1}) 
viterbi: defaultdict(<class 'float'>, {1: 1})
my f2e: [[1.]] 
 my e2f: [[1.]] 
right: [[1]] 
-----
['the', 'the', 'house', 'a', 'book'] ['das', 'ein', 'Buch', 'Haus'] 
viterbi e2f: defaultdict(<class 'float'>, {1: 1, 2: 1, 3: 4, 4: 2, 5: 3}) 
viterbi: defaultdict(<class 'float'>, {1: 1, 2: 4, 3: 5, 4: 3})
my f2e: [[1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]] 
 my e2f: [[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]] 
right: [[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0]] 
-----
['book', 'a'] ['Buch', 'ein'] 
viterbi e2f: defaultdict(<class 'float'>, {1: 1, 2: 2}) 
viterbi: defaultdict(<class 'float'>, {1: 1, 2: 2})
my f2e: [[1. 0.]
 [0. 1.]] 
 my e2f: [[1. 0.]
 [0. 1.]] 
right: [[1, 0], [0, 1]] 
-----
['book', 'the'] ['Buch', 'ein'] 
viterbi e2f: defaultdict(<class 'float'>, {1: 1, 2: 1}) 
viterbi: defaultdict(<class 'float'>, {1: 1, 2: 1})
my

In [50]:
#TODOS

#do summary 09!

#TODO unaligned cases!

#debug ibm 2
#IBM Model 2: How to calculate a correctly?
#implementation exactly like in book
#eg sum is supposed to always be 1
#use ibm model 2 in slides
#compare subresults if they are right, count, stotal etc
#look into indexing, starting 0 / 1?

# print(φ(['house'], ['Haus'], count))
# print(φ(['the','house'],['das','Haus']))
# doesnt work

# TODO translation probability is not working for phrases containing several words
#p(e|f) for IBM Model 2: Gives result 4 should be 1?
#is sum wrong maybe?
#> compare with other implementations

#decoding
#generate some example target sentences (word sequences) “manually”
#and calculating p(e|f) for only that, & use t for that?
#make up f, dont implement max, just some examples

#calculate Viterbi alignment for ibm2

#look into efficiency issues, use hashing?
# TODO For the estimation of the phrase translation probabilities, not all
# phrase pairs have to be loaded into memory. It is possible to efficiently
# estimate the probability distribution by storing and sorting the extracted
# phrases on disk. Similarly, when using the translation table for the translation of a single sentence, only a small fraction of it is needed and may
# be loaded on demand.
# think about saving stuff in file
# • Phrase translation table typically bigger than corpus
# ... even with limits on phrase lengths (e.g., max 7 words)
# → Too big to store in memory?
# • Solution for training
# – extract to disk, sort, construct for one source phrase at a time
# • Solutions for decoding
# – on-disk data structures with index for quick look-ups
# – suffix arrays to create phrase pairs on demand
# TODO use numpy for everything
# maybe other data structure ['the','house']:[['das','Haus'],['ein','Haus']]
# faster sometimes but slower if translation in other direction